In [1]:
# Import required modules
from copy import copy

import numpy as np
import time

from lcode2dPy.plasma3d.initialization import init_plasma
from lcode2dPy.push_solver_3d import PushAndSolver3d

from lcode2dPy.beam3d import beam
from lcode2dPy.beam3d.beam_generator import *

from lcode2dPy.config.default_config import default_config

In [2]:
# Create a config
config = copy(default_config)

config.set('geometry', '3d')

config.set('window-width-steps', 501)
config.set('window-width-step-size', 0.02)
config.set('window-width', 501*0.02)

config.set('window-length', 5)
config.set('xi-step', 0.02)

config.set('time_limit', 100.5)
config.set('time_step', 25)

# Make ready a pusher-solver
pusher_solver = PushAndSolver3d(config)

In [3]:
# Create a beam
geometry = '3d'

angspread = 1e-5
m_proton = 958/0.51
Ipeak_kA = 40/1000
gamma = 426 * m_proton # Why m_proton?

init_beam = make_beam(config,
                      xi_distr = Gauss(vmin=-10, vmax=0),
                      x_distr  = Gauss(vmin=-7.5, vmax=7.5),
                      px_distr = Gauss(sigma=gamma*angspread, vmin=None, vmax=None),
                      pz_distr = Gauss(gamma, gamma*1e-4, vmin=None, vmax=None),
                      Ipeak_kA = Ipeak_kA, q_m = 1 / m_proton,
                      partic_in_layer=1000)

np.savez_compressed('beamfile',
                    xi = init_beam['xi'].to_numpy(),
                    x = init_beam['x'].to_numpy(),
                    y = init_beam['y'].to_numpy(),
                    p_x = init_beam['px'].to_numpy(),
                    p_y = init_beam['py'].to_numpy(),
                    p_z = init_beam['pz'].to_numpy(),
                    q_m = init_beam['q_m'].to_numpy(),
                    q_norm = init_beam['q'].to_numpy(),
                    id = init_beam['N'].to_numpy('int'))

Number of particles: 125331
Number of particles in the middle layer: 999


In [4]:
# Load the beam
beam_particles = beam.BeamParticles(0)
beam_particles.load('beamfile.npz')
beam_calulator = beam.BeamCalculator(config, beam_particles)

In [5]:
# Simulation loop
start_time = time.time()
time_limit = config.getfloat('time-limit')
time_step_size = config.getfloat('time-step')
time_steps = int(time_limit / time_step_size)

for t_i in range(time_steps + 1):
    pl_fields, pl_particles, pl_currents, pl_const_arrays = init_plasma(config)

    pusher_solver.step_dt(pl_fields, pl_particles, pl_currents, pl_const_arrays,
                          beam_particles, beam_calulator)

print("--- %s seconds ---" % (time.time() - start_time))

Plasma solver done in +4.2583 s.	Beam pusher done in +0.0001 s.
xi=+0.0000 -6.9874e-06
Plasma solver done in +1.8120 s.	Beam pusher done in +0.6585 s.
xi=+0.0200 -2.0891e-05
Plasma solver done in +1.2263 s.	Beam pusher done in +0.1207 s.
xi=+0.0400 -3.4627e-05
Plasma solver done in +1.2385 s.	Beam pusher done in +0.1138 s.
xi=+0.0600 -4.8211e-05
Plasma solver done in +1.2220 s.	Beam pusher done in +0.1140 s.
xi=+0.0800 -6.2070e-05
Plasma solver done in +1.2113 s.	Beam pusher done in +0.1177 s.
xi=+0.1000 -7.5685e-05
Plasma solver done in +1.2197 s.	Beam pusher done in +0.1147 s.
xi=+0.1200 -8.9506e-05
Plasma solver done in +1.2193 s.	Beam pusher done in +0.1149 s.
xi=+0.1400 -1.0310e-04
Plasma solver done in +1.2265 s.	Beam pusher done in +0.1154 s.
xi=+0.1600 -1.1643e-04
Plasma solver done in +1.2248 s.	Beam pusher done in +0.1140 s.
xi=+0.1800 -1.2990e-04
Plasma solver done in +1.2269 s.	Beam pusher done in +0.1147 s.
xi=+0.2000 -1.4272e-04
Plasma solver done in +1.2375 s.	Beam pushe

KeyboardInterrupt: 